In [43]:
## Import data for Splitty or Not Splitty training data -------------------
import numpy as np
import pandas as pd

%cd C:/Users/David/Google Drive/CSC478Proj/processing data2
#%cd C:/Users/David/Dropbox/DePaulUniv/CSC478/HW2

print
splitty_df = pd.read_csv("splitty_rows.csv", sep=',', header=0, na_values=[''], index_col=0)  #yes for headers
#yarn_all = pd.read_csv("all_rows_all_cols_reduced.csv", sep=',', header=0, na_values=[''], index_col=0)
notsplitty_df = pd.read_csv("not_splitty_rows.csv", sep=',', header=0, na_values=[''], index_col=0) 
yarn_all = pd.read_csv("all_reduced_reg_norm.csv", sep=',', header=0, na_values=[''], index_col=0)

C:\Users\David\Google Drive\CSC478Proj\processing data2



In [44]:
print 'splitty ', splitty_df.shape
print 'not splitty ', notsplitty_df.shape
print 'all yarn ', yarn_all.shape

yarn_all_np = np.array(yarn_all)
splitty_np = np.array(splitty_df)
notsplitty_np = np.array(notsplitty_df)

splitty  (134, 151)
not splitty  (133, 151)
all yarn  (3657, 151)


#### Training data distribution Splitty = 134 / (134 + 133) = 50.2%  and Not Pilly = 133 / (134 + 133)  = 50%

In [46]:
#print splitty_np[0:5]
#np.ones(splitty_np.shape[0]) * 2

#### Func to create new prototype Rocchio Vector with given dataset

In [47]:
def AddRocchioV(PVector, classdata, cvalue):
    #expect a numpy Prototype Vector to pass in for appending
    #a dataset with given class 'cvalue' to be computed
    #create empty class vector of ones to be populated with cvalue
    classvector = np.ones(classdata.shape[0])
    classvector =  classvector * cvalue
        
    NumRows = classdata.shape[0]
    
    for row in range(NumRows):
    #for row in range(5):         #Testing with 10 only
        PVector[cvalue] = PVector[cvalue] +  classdata[row]  
    
    #check out prototype vectors
    #for v in PVector:
    print 'Contents of Prototype Vector for Class ', cvalue
    print PVector[cvalue] [0:5]
        
    return PVector, classvector

In [48]:
#create numpy array of class labels
yarn_classes = [0, 1]
yclass_np = np.array(yarn_classes)
print yclass_np[0]
print yclass_np[1]

0
1


In [49]:
## initialize 0 vectors for Prototype Vectors with rows = Number of Classes
pV = np.zeros( (2 , yarn_all.shape[1]), dtype=np.float )
print 'shape of Prototype Vectors' , pV.shape
#print pV[0]
#print pV[1]


shape of Prototype Vectors (2L, 151L)


In [50]:
## create not splitty Prototype Vector using class 0
pV, Y_notsplitty = AddRocchioV(pV, notsplitty_np, 0)

Contents of Prototype Vector for Class  0
[ 12.   3.  90.   1.   4.]


In [51]:
## create splitty Prototype Vector using class 1
pV, Y_splitty = AddRocchioV(pV, splitty_np, 1)

Contents of Prototype Vector for Class  1
[ 15.   0.  94.   5.   1.]


In [52]:
print Y_notsplitty.shape
print Y_splitty.shape
Y_pilly[0:5]

(133L,)
(134L,)


array([ 1.,  1.,  1.,  1.,  1.])

In [56]:
print 'Prototype Vectors created for Not Splitty 0 and Splitty 1'
print pV[0][0:10]
print pV[1][0:10]

Prototype Vectors created for Not Splitty 0 and Splitty 1
[ 12.   3.  90.   1.   4.   1.   3.   4.   0.   3.]
[ 15.   0.  94.   5.   1.   0.   0.   0.   1.   1.]


In [57]:
def RocCoSCategorize(testdf, testclasses, protoV, uniqclasses):
    #Rocchio Text Categorization, prototype vectors for CoSine similarity required
    
    if len(protoV) <> len(uniqclasses):
        print 'Error!  # of Prototype Vectors do not match # of Unique Classes'
        
    numTestrows = len(testclasses)
    testclasses = np.array(testclasses)
    testdf      = np.array(testdf)
    errorCount = 0.0
    resultlist = []
    cosinelist=[]

    for row in range(numTestrows):
    #for row in range(5):         #Testing with 10 only
        
        currCoS = np.float(-2.0)   #init with max value for each new row
        D = testdf[row]             #doc vector for categorization
        cosine_sim = 0.000
                
        for iclass in range(len(protoV)):
            p = protoV[iclass]          #prototype vector to be compared to
            D_norm = np.linalg.norm(D)   #normalize test doc vector curr row
            p_norm = np.linalg.norm(p)
            cosine_sim = np.dot(D,p) / (D_norm * p_norm)
            #print 'curr cosine_sim for row ', row, cosine_sim
            if cosine_sim > currCoS:
                currCoS = cosine_sim
                classfound = uniqclasses[iclass]
                
        
        #store results
        resultlist.append( [classfound, testclasses[row] ]  ) 
        cosinelist.append(currCoS)
        #print 'Test Row', row, 'was classified Euclidean as %s, actual answer is %s' % (votedKNN, classTest['class'][row])
        if (classfound !=  testclasses[row]): errorCount += 1.0

    errorRate = errorCount/float(numTestrows)
    #print "the Rocchio total error rate is: %f" % (errorRate)
    return errorRate, resultlist, np.array(cosinelist)

In [87]:
## call Rocchio function 
RocchErrRate, RocSplitty, SplittyCoSign = RocCoSCategorize(splitty_np, Y_splitty, pV, yclass_np)
print "the Rocchio Splitty total error rate is: %f" % (RocchErrRate)
#print 'Rocchio Error results is ', Rocresults 

the Rocchio Splitty total error rate is: 0.305970


In [68]:
print SplittyCoSign.shape
len(RocSplitty)

(134L,)


134

In [89]:
print 'Results for Splitty'
print "the Rocchio total error rate is: %f" % (RocchErrRate)
print "the Rocchio Accuracy rate is: %f" % (1 - RocchErrRate)

Results for Splitty
the Rocchio total error rate is: 0.305970
the Rocchio Accuracy rate is: 0.694030


#### Look at CoSign value distributions of Splitty vs Not Splitty

In [90]:
SplittyCoS_df = pd.DataFrame( SplittyCoSign)
SplittyCoS_df.describe(include="all")

0
count  134.000000
mean     0.562864
std      0.105352
min      0.256303
25%      0.489283
50%      0.570175
75%      0.641523
max      0.766362

In [91]:
## call Rocchio function Not Splitty
RocchErrRate, RocNotSplitty, notSplittyCoSign = RocCoSCategorize(notsplitty_np, Y_notsplitty, pV, yclass_np)
print "the Rocchio Not Splittytotal error rate is: %f" % (RocchErrRate)
#print 'Rocchio Error results is ', Rocresults 

the Rocchio Not Splittytotal error rate is: 0.270677


In [92]:
print 'Results for Not Splitty'
print "the Rocchio total error rate is: %f" % (RocchErrRate)
print "the Rocchio Accuracy rate is: %f" % (1 - RocchErrRate)

Results for Not Splitty
the Rocchio total error rate is: 0.270677
the Rocchio Accuracy rate is: 0.729323


In [72]:
notSplittyCoS_df = pd.DataFrame( notSplittyCoSign)
notSplittyCoS_df.describe(include="all")

0
count  133.000000
mean     0.539367
std      0.102299
min      0.254148
25%      0.455051
50%      0.547278
75%      0.617044
max      0.756734

In [73]:
print 'Results for not Splitty'
print "the Rocchio total error rate is: %f" % (RocchErrRate)
print "the Rocchio Accuracy rate is: %f" % (1 - RocchErrRate)

Results for not Splitty
the Rocchio total error rate is: 0.270677
the Rocchio Accuracy rate is: 0.729323


#### Confusion Matrix on Splitty label results using 75% IQR level as cutoff for estimating True Positives

In [93]:
print SplittyCoSign[0:3]
count_PAll = SplittyCoSign > 0.64
#print count_PAll[0:5]
my_tab = pd.crosstab(index=count_PAll ,   columns="count",  margins=True  ) #  # Name the count column
print my_tab
print 'Splitty Accuracy counting Cosign > 0.500 ', 
my_tab/my_tab.ix["All"]

[ 0.4445529   0.61101755  0.76636187]
col_0  count  All
row_0            
False     99   99
True      35   35
All      134  134
Splitty Accuracy counting Cosign > 0.500 

col_0     count       All
row_0                    
False  0.738806  0.738806
True   0.261194  0.261194
All    1.000000  1.000000

#### Confusion Matrix on Not Splitty label results 75% IQR level as cutoff for estimating True Positives

In [94]:
print notSplittyCoSign[0:3]
count_PAll = notSplittyCoSign > 0.61
#print count_PAll[0:5]
my_tab = pd.crosstab(index=count_PAll ,   columns="count",  margins=True  ) #  # Name the count column
print my_tab
print 'Not Pilly Accuracy counting Cosign > 0.500 ', 
my_tab/my_tab.ix["All"]

[ 0.57897914  0.55542732  0.46848552]
col_0  count  All
row_0            
False     96   96
True      37   37
All      133  133
Not Pilly Accuracy counting Cosign > 0.500 

col_0     count       All
row_0                    
False  0.721805  0.721805
True   0.278195  0.278195
All    1.000000  1.000000

#### Try Rocchio on ALL Yarn

In [95]:
def RocCoSAllYarn(testdf, protoV, uniqclasses):
    #Rocchio Text Categorization, prototype vectors for CoSine similarity required
    
    if len(protoV) <> len(uniqclasses):
        print 'Error!  # of Prototype Vectors do not match # of Unique Classes'
      
    testdf      = np.array(testdf)
    numTestrows = len(testdf)
    errorCount = 0.0
    resultlist = []
    cosinelist=[]

    for row in range(numTestrows):
    #for row in range(5):         #Testing with 10 only
        
        currCoS = np.float(-2.0)   #init with max value for each new row
        D = testdf[row]             #doc vector for categorization
                
        for iclass in range(len(protoV)):
            p = protoV[iclass]          #prototype vector to be compared to
            D_norm = np.linalg.norm(D)   #normalize test doc vector curr row
            p_norm = np.linalg.norm(p)
            cosine_sim = np.dot(D,p) / (D_norm * p_norm)
            #print 'curr cosine_sim for row ', row, cosine_sim
            if cosine_sim > currCoS:
                currCoS = cosine_sim
                classfound = uniqclasses[iclass]
        
        #store results
        resultlist.append( classfound ) 
        cosinelist.append(currCoS)
        
    return resultlist, np.array(cosinelist)

In [96]:
## call Rocchio function All Yarn
RocAllYarn , AllYarnCoSign = RocCoSAllYarn(yarn_all_np , pV, yclass_np)

In [97]:
AllYarnCoS_df = pd.DataFrame(AllYarnCoSign, columns=[ 'CoSign' ])
AllYarnCoS_df.describe(include="all")

CoSign
count  3657.000000
mean      0.521139
std       0.121952
min       0.124678
25%       0.435028
50%       0.538052
75%       0.611195
max       0.772732

In [98]:
AllYarnCoSign.shape

(3657L,)

#### Confusion Matrix on All Yarn results & use 75% IQR level as cutoff for estimating True Positives

In [99]:
print AllYarnCoSign[0:3]

count_PAll = AllYarnCoSign > 0.611000
#print count_PAll[0:5]
my_tab = pd.crosstab(index=count_PAll ,   columns="count",  margins=True  ) #  # Name the count column
print my_tab
print 
print 'All Yarn Roccio Splitty Accuracy counting Cosign > 0.611 ', 
my_tab/my_tab.ix["All"]


[ 0.57897914  0.62538521  0.66955792]
col_0  count   All
row_0             
False   2740  2740
True     917   917
All     3657  3657

All Yarn Roccio Splitty Accuracy counting Cosign > 0.611 

col_0     count       All
row_0                    
False  0.749248  0.749248
True   0.250752  0.250752
All    1.000000  1.000000

In [83]:
AllYarnCoS_df['yarn_id'] = yarn_all.index

In [84]:
AllYarnCoS_df.shape

(3657, 2)

In [85]:
import csv
import os
currentPath = os.getcwd()
print currentPath
%cd C:/Users/David/Google Drive/CSC478Proj/predict1

C:\Users\David\Google Drive\CSC478Proj\processing data2
C:\Users\David\Google Drive\CSC478Proj\predict1


In [86]:
AllYarnCoS_df.to_csv('Rocchio Cosign Splitty All Yarn All Features_June5.csv', index=False)